<a href="https://colab.research.google.com/github/kryuchkovdm/Distillation/blob/master/models/LinearPrune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class LinearModelPrune(nn.Module):
    
    def __init__(self, n_vocab, n_labels, embedding_dim, padding_idx):
        super(LinearModelPrune, self).__init__()
        self.embeddings = nn.Embedding(n_vocab, embedding_dim, padding_idx=padding_idx)
        self.output = nn.Linear(embedding_dim, n_labels)
        with torch.no_grad():
            self.embeddings.weight.uniform_(to=1.0 / embedding_dim)
            self.embeddings.weight[padding_idx] = 0            
            self.output.weight.zero_()

    def forward(self, input_ids):
        """Only input ids are required - kwargs are for API compat with BERT."""
        
        X = self.embeddings(input_ids)
        X = X.mean(dim=1)
        X = self.output(X)
        return X

    def prune(self, rate):
        prune.l1_unstructured(self.output, 'weight', amount=rate)